In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



Load the Dataset

In [11]:

# Load transition matrix
transition_df = pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\transition_probability_matrix.csv", index_col=0)

# Load emission matrix
emission_df = pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\emissive_probability_matrix.csv", index_col=0)

# Ensure index and column names are properly set
print("Transition Matrix Index:", transition_df.index)
print("Transition Matrix Columns:", transition_df.columns)



# transition_df=pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\transition_probability_matrix.csv")
# emission_df=pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\emissive_probability_matrix.csv")
start_df=pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\start_probability_matrix.csv")

Transition Matrix Index: Index(['NUM', 'DET', 'VERB', 'PROPN', 'PRON', 'NOUN', 'ADP', 'AUX', '.', 'PRT',
       'CCONJ', 'INTJ', 'ADV', 'SCONJ', 'CONJ', '_', 'PUNCT', 'X', 'ADJ',
       'SYM', 'PART'],
      dtype='object')
Transition Matrix Columns: Index(['NUM', 'DET', 'VERB', 'PROPN', 'PRON', 'NOUN', 'ADP', 'AUX', '.', 'PRT',
       'CCONJ', 'INTJ', 'ADV', 'SCONJ', 'CONJ', '_', 'PUNCT', 'X', 'ADJ',
       'SYM', 'PART'],
      dtype='object')


In [12]:
print("Transition Matrix Index:", transition_df.index)
print("Transition Matrix Columns:", transition_df.columns)
print("Emission Matrix Index:", emission_df.index)
print("Emission Matrix Columns:", emission_df.columns)


Transition Matrix Index: Index(['NUM', 'DET', 'VERB', 'PROPN', 'PRON', 'NOUN', 'ADP', 'AUX', '.', 'PRT',
       'CCONJ', 'INTJ', 'ADV', 'SCONJ', 'CONJ', '_', 'PUNCT', 'X', 'ADJ',
       'SYM', 'PART'],
      dtype='object')
Transition Matrix Columns: Index(['NUM', 'DET', 'VERB', 'PROPN', 'PRON', 'NOUN', 'ADP', 'AUX', '.', 'PRT',
       'CCONJ', 'INTJ', 'ADV', 'SCONJ', 'CONJ', '_', 'PUNCT', 'X', 'ADJ',
       'SYM', 'PART'],
      dtype='object')
Emission Matrix Index: Index(['NUM', 'NOUN', 'DET', 'VERB', 'ADJ', '.', 'PUNCT', 'ADP', 'CONJ',
       'PROPN', 'X', 'PRON', 'ADV', 'AUX', 'INTJ', 'CCONJ', 'PRT', 'PART',
       'SCONJ', '_', 'SYM'],
      dtype='object')
Emission Matrix Columns: Index(['03/01/2001', '01:35', 'PM', 'The', 'industry', 'has', 'said', '1960',
       'was', 'a',
       ...
       'computations', 'mea', 'culpas', 'scapegoats', 'slave-owners', 'MKM's',
       '*-150', '*-151', 'mosquito-plagued', 'Superstition'],
      dtype='object', length=58110)


In [13]:
def viterbi_algorithm(sentence, transition_df, emission_df):
    pos_tags = list(transition_df.index)  # List of POS tags
    words = sentence.split()
    num_tags = len(pos_tags)
    num_words = len(words)
    
    # Initialize dynamic programming tables
    viterbi = np.zeros((num_tags, num_words))  # Probability table
    backpointer = np.zeros((num_tags, num_words), dtype=int)  # Backtracking table
    
    # Initialize the first column
    for i, tag in enumerate(pos_tags):
        transition_prob = transition_df.loc['START', tag] if 'START' in transition_df.index else 1e-6
        emission_prob = emission_df.loc[tag, words[0]] if words[0] in emission_df.columns else 1e-6
        viterbi[i, 0] = transition_prob * emission_prob
        backpointer[i, 0] = 0
    
    # Fill the dynamic programming table
    for t in range(1, num_words):
        for i, tag in enumerate(pos_tags):
            max_prob, max_state = max(
                (viterbi[k, t-1] * transition_df.loc[prev_tag, tag] if prev_tag in transition_df.index else 1e-6 *
                 emission_df.loc[tag, words[t]] if words[t] in emission_df.columns else 1e-6, k)
                for k, prev_tag in enumerate(pos_tags)
            )
            viterbi[i, t] = max_prob
            backpointer[i, t] = max_state
    
    # Traceback
    best_path = []
    best_last_state = np.argmax(viterbi[:, -1])
    best_path.append(pos_tags[best_last_state])
    
    for t in range(num_words - 1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, pos_tags[best_last_state])
    
    return list(zip(words, best_path))

# Example usage
sentence = "The industry has said"
predicted_tags = viterbi_algorithm(sentence, transition_df, emission_df)
print(predicted_tags)



[('The', 'NOUN'), ('industry', 'ADP'), ('has', 'DET'), ('said', 'NOUN')]
